PDF Text extraction

In [20]:
import os
from openai import OpenAI
from settings import *
import fitz
import gradio as gr

# os.environ["GRADIO_ANALYTICS_ENABLED"] = "False"

In [ ]:
#NOT BEING USED


# def get_db_connection():
#     try:
#         print("Attempting to connect to database...")
#         print("Host: mysql-samadhan-parakhc4.b.aivencloud.com, Port: 12945")
#         connection = pymysql.connect(
#             charset="utf8mb4",
#             connect_timeout=10,
#             cursorclass=pymysql.cursors.DictCursor,
#             db="doctors_database",
#             host="mysql-samadhan-parakhc4.b.aivencloud.com",
#             password="AVNS_iy-fKKvQNt-u1-gHDcW",
#             read_timeout=10,
#             port=12945,
#             user="avnadmin",
#             write_timeout=10,
#         )
#         print("Database connection successful.")
#         return connection
#     except Exception as e:
#         print(f"Error connecting to the database: {e}")
#         raise
    
# conn = sqlite3.connect('doctors.db')
# cursor = conn.cursor()

In [ ]:
#Testing report text extraction functionality
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text.strip()

pdf_text = extract_text_from_pdf("cbc-report-format.pdf")
print(pdf_text[:50])


HAEMATOLOGY
COMPLETE BLOOD COUNT (CBC)
TEST
VALUE



In [22]:
def domain_specific_intent_restriction(generated_text):
    message = [
            {"role": "system", "content": "You are an AI assistant skilled in verifying whether provided content is within restricted boundary"},
            {"role": "user", "content": f"A chat Agent has generated the following summary of a PDF file. Please verify if the summary is of a Medical Health report. Reply with Yes or No only.:\n\n{generated_text}"}
        ]
    response = monster_client.chat.completions.create(
        model=monster_ai_model_name[llm_name],
        messages=message,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        stream=False
    )

    binary_classifier = response.choices[0].message.content

    if binary_classifier == "Yes":
        return True
    else:
        return False


In [23]:
def chat_with_llm(user_input, file=None):
    try:
        # If a file is uploaded, extract text
        pdf_text = ""
        if file:
            pdf_text = extract_text_from_pdf(file.name)
            if not pdf_text.strip():
                return "The uploaded PDF appears to be empty. Please try with a valid medical report."

        # Create context based on input and optional PDF content
        context = f"Medical report content: {pdf_text}" if file else ""
        message = [
            {"role": "system", "content": "You are an AI assistant skilled in answering questions about medical reports."},
            {"role": "user", "content": f"{context}\n\n{user_input}"}
        ]

        # Generate response using Monster API
        response = monster_client.chat.completions.create(
            model=monster_ai_model_name[llm_name],
            messages=message,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
            stream=False
        )

        # Extract the chat response
        generated_text = response.choices[0].message.content
        return generated_text

    except Exception as e:
        return f"Error occurred: {e}"

In [ ]:
# from qdrant_client import models, QdrantClient
# from sentence_transformers import SentenceTransformer

# # Initialize the encoder and Qdrant client
# encoder = SentenceTransformer("all-MiniLM-L6-v2")
# client = QdrantClient(":memory:")  # For in-memory use; replace with cluster host info for persistent storage


# collection_name = "medical_reports"
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config=models.VectorParams(
#         size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by the encoder
#         distance=models.Distance.COSINE,  # Use cosine similarity
#     ),
# )

# def chunk_text(text, chunk_size=500):
#     return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# chunks = chunk_text(pdf_text)

# client.upload_points(
#     collection_name=collection_name,
#     points=[
#         models.PointStruct(
#             id=idx,
#             vector=encoder.encode(chunk).tolist(),
#             payload={"chunk": chunk}
#         )
#         for idx, chunk in enumerate(chunks)
#     ],
# )

# print(f"Stored {len(chunks)} chunks in Qdrant.")


# def query_relevant_chunks(query_text, top_k=5):
#     query_vector = encoder.encode(query_text).tolist()
#     hits = client.query_points(
#         collection_name=collection_name,
#         query=query_vector,
#         limit=top_k,
#     ).points
#     return [hit.payload["chunk"] for hit in hits]

# query = "summarize the medical findings"
# relevant_chunks = query_relevant_chunks(query)

# print("Relevant chunks for summarization:")
# for chunk in relevant_chunks:
#     print(chunk[:200], "...")



In [ ]:
# from openai import OpenAI
# from settings import *
# import os

# generation_model_name: str
# temperature: float = 0.9
# top_p = 0.9
# max_tokens: int = 2048
# stream: bool = True
# llm_name: str = "Meta-Llama"

# monster_client = OpenAI(
#     base_url="https://llm.monsterapi.ai/v1/",
#     api_key=str(MONSTER_API_KEY)
# )

# monster_ai_model_name = {
#     "Google-Gemma": "google/gemma-2-9b-it",
#     "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
#     "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
#     "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
# }

# # Context and Summarization Prompt
# message = [
#     {"role": "system", "content": "You are an AI assistant skilled in summarizing complex medical reports."},
#     {"role": "user", "content": f"Summarize the following medical report in simple terms. Focus on the key findings, diagnoses, and recommendations. Report text:\n\n{pdf_text}"}
# ]

# # Generate Summary
# response = monster_client.chat.completions.create(
#     model=monster_ai_model_name[llm_name],
#     messages=message,
#     temperature=temperature,
#     top_p=top_p,
#     max_tokens=max_tokens,
#     stream=stream
# )

# # Collect Generated Text
# generated_text = ""
# for chunk in response:
#     if chunk.choices[0].delta.content is not None:
#         generated_text += chunk.choices[0].delta.content

# print("Generated Summary:")
# print(generated_text)



In [28]:
SYMPTOM_SPECIALTY_MAP = {
    "abdominal pain": "Gastroenterologist",
    "pregnancy": "Gynecologist",
    "fever": "General Physician",
    "chest pain": "Cardiologist",
    "skin rash": "Dermatologist",
    "depression" : "Psychiatrist",
    "headache" : "Neurologist",
}

def get_doctors_by_specialty(specialty):
    try:
        connection = get_db_connection()
        with connection.cursor() as cursor:

            query = """
                SELECT id, name, specialization, latitude, longitude, address, contact 
                FROM doctors 
                WHERE LOWER(specialization) = LOWER(%s)
            """

            cleaned_specialty = specialty.strip()
            print(f"Executing query for: '{cleaned_specialty}'")  # Debugging

            cursor.execute(query, (cleaned_specialty,))
            doctors = cursor.fetchall()
            print("Query result:", doctors)  # Debugging output
        connection.close()

        return doctors if doctors else []
    except Exception as e:
        return f"Error fetching doctors: {e}"




# def get_doctors_by_specialty(specialty):
#     try:
#         connection = get_db_connection()
#         with connection.cursor() as cursor:
#             query = """
#                 SELECT id, name, specialization, latitude, longitude, address, contact 
#                 FROM doctors 
#                 WHERE LOWER(specialization) = LOWER(%s)
#             """

#             cursor.execute(query, (specialty,))
#             doctors = cursor.fetchall()
#             print("Query result:", doctors)  # Debugging output
#         connection.close()

#         return doctors if doctors else []
#     except Exception as e:
#         return f"Error fetching doctors: {e}"

def recommend_doctors(symptoms):

    specialty = None
    for symptom, spec in SYMPTOM_SPECIALTY_MAP.items():
        if symptom in symptoms.lower():
            specialty = spec
            break
    
    print("Inferred specialty:", specialty)  # Debugging

    if not specialty:
        return "Sorry, we couldn't infer a relevant specialty for your symptoms. Please provide more details."
    specialty = specialty.strip()


    doctors = get_doctors_by_specialty(specialty)
    print("Doctors fetched:", doctors)  # Debugging output

    if not doctors:
        return f"No doctors found for the specialty: {specialty}."

    return "\n\n".join(
        f"Dr. {doc['name']} ({doc['specialization']})\nAddress: {doc['address']}\nContact: {doc['contact']}" 
        for doc in doctors
    )




In [ ]:

# PDF Text Extraction 
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text.strip()


generation_model_name: str
temperature: float = 0.9
top_p = 0.9
max_tokens: int = 2048
stream: bool = False  
llm_name: str = "Meta-Llama"

monster_client = OpenAI(
    base_url="https://llm.monsterapi.ai/v1/",
    api_key=str(MONSTER_API_KEY)
)

monster_ai_model_name = {
    "Google-Gemma": "google/gemma-2-9b-it",
    "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
    "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
    "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
}

# Summarization Function
def summarize_pdf(file):
    try:
        if not file.name.endswith(".pdf"):
            return "Invalid file format. Please upload a PDF."


        pdf_text = extract_text_from_pdf(file.name)
        if not pdf_text.strip():
            return "The uploaded PDF appears to be empty. Please try with a valid medical report."


        message = [
            {"role": "system", "content": "You are an AI assistant skilled in summarizing complex medical reports. Strict instruction: Only summarise a report if it is a medical report. Otherwise, reply with 'Attached PDF is not a medical report'"},
            {"role": "user", "content": f"Summarize the following medical report in simple terms. Focus on the key findings, diagnoses, and recommendations. Report text:\n\n{pdf_text}"}
        ]


        response = monster_client.chat.completions.create(
            model=monster_ai_model_name[llm_name],
            messages=message,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
            stream=False
        )

        # Extract the summary from the response
        generated_text = response.choices[0].message.content

        # validation 
        return generated_text

    except Exception as e:
        return f"Error occurred: {e}"



# Gradio Interface
# interface = gr.Interface(
#     fn=chat_with_llm,
#     inputs=[
#         gr.Textbox(lines=2, placeholder="Type your question here...", label="Ask a Question"),
#         gr.File(label="Optional: Upload your Medical Report (PDF)")
#     ],
#     outputs=gr.Textbox(label="AI's Response"),
#     title="Medical Report Chat Assistant",
#     description=(
#         "Ask questions about your medical report or any medical context. "
#         "Optionally, upload a PDF report for more tailored answers."
#     )
# )

# Gradio UI
with gr.Blocks() as interface:
    gr.Markdown("# 🥼 Samadhan Conversational HealthBot 1.0")
    gr.Markdown("Upload a medical report or describe your symptoms to get recommendations.")

    with gr.Tab("🩻 Summarize Report"):
        file_input = gr.File(label="Upload your Medical Report (PDF)")
        summary_output = gr.Textbox(label="Summary")
        summarize_btn = gr.Button("Summarize")
        summarize_btn.click(summarize_pdf, inputs=file_input, outputs=summary_output)

    with gr.Tab("💬 Chat with AI"):
        chat_input = gr.Textbox(lines=2, placeholder="Ask a question about your medical report...", label="Your Question")
        chat_file = gr.File(label="Optional: Upload Medical Report")
        chat_output = gr.Textbox(label="AI's Response")
        chat_btn = gr.Button("Ask")
        chat_btn.click(chat_with_llm, inputs=[chat_input, chat_file], outputs=chat_output)

    with gr.Tab("🩺 Find a Doctor"):
        symptoms_input = gr.Textbox(lines=3, placeholder="Describe your symptoms...", label="Symptoms")
        doctor_output = gr.Textbox(label="Recommended Doctors")
        doctor_btn = gr.Button("Find Doctors")
        doctor_btn.click(recommend_doctors, inputs=symptoms_input, outputs=doctor_output)


interface.launch(share=True)



* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a177c873e7cda69cb1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
